# Time Stamp Differences
 Looking into the difference between sent_at (reported by the browser) and received_at (reported by the browser).
 
In the ETL we're looking at sent at, 

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
import numpy as np

In [ ]:
from athena_querying import AthenaQuery, athena_database, athena_raw_events_table

In [ ]:
from athena_common_queries import create_partition_filter

In [ ]:
import plotly.express as px

In [ ]:
import plotly.graph_objects as go

# Settings

In [ ]:
num_days_to_query = 10
to_datetime = datetime(year=2020, month=2, day=9)
from_datetime = to_datetime - timedelta(days=num_days_to_query)
limit_query = False

# Test plotting

In [ ]:
#px.scatter(x=np.arange(10), y=np.random.randn(10))

# Loading Data

In [ ]:
aq = AthenaQuery()

In [ ]:
aq.connect()

In [ ]:
#NB this doesn't filter by date, only partition, which is a proxy.  Expect some outliars (but it's likely good to see them)

query = "select event_id, received_at, sent_at from "+athena_database + "."+ athena_raw_events_table

query+= " where "
query+= create_partition_filter(from_datetime, to_datetime) 
if limit_query:
    
    query+=" limit 1000"

In [ ]:
print(query)

In [ ]:
df = aq.query(query)

In [ ]:
print("got %i records"%len(df))

In [ ]:
df["sent"] = pd.to_datetime(df["sent_at"])
df["received"] = pd.to_datetime(df["received_at"])

In [ ]:
df.columns

In [ ]:
df["received"].head()

In [ ]:
df["dt"] = (pd.to_datetime(df["sent"]) - pd.to_datetime(df["received"])).dt.total_seconds()

# Timedelta vs Sent_at time

# Timedelta Distribution

In [ ]:
threshold_for_plot_seconds = 5*60

In [ ]:
num_more_than_threshold_positive = len(df[df.dt>threshold_for_plot_seconds])
num_more_than_threshold_negative = len(df[df.dt<(-1*threshold_for_plot_seconds)])
total_events = len(df)
num_within_threshold = (total_events - num_more_than_threshold_positive - num_more_than_threshold_negative)
percentage_pos = num_more_than_threshold_positive / total_events
percentage_neg = num_more_than_threshold_negative / total_events
percentage_norm = num_within_threshold / total_events


In [ ]:
pie_labels = ["Within %i s"%threshold_for_plot_seconds, "sent at a lot greater", "received_at a lot greater"]
pie_values = [num_within_threshold, num_more_than_threshold_positive, num_more_than_threshold_negative]

In [ ]:
#px.pie(values = pie_values, 
#       names = pie_labels)

In [ ]:
fig = go.Figure(data =[go.Pie(labels=pie_labels, values=pie_values)])
fig.show()

In [ ]:
dts_in_threshold = df[(df.dt<=threshold_for_plot_seconds) & (df.dt>=(-1*threshold_for_plot_seconds))]["dt"]

In [ ]:
len(dts_in_threshold)

In [ ]:
dts_in_threshold.head()

In [ ]:
dts_in_threshold.describe()

In [ ]:
px.histogram(dts_in_threshold,x="dt", nbins=500) #if you don't set the bins,it seems to create endless ones